# 0. 기본 정보
* 코드 작성자: 여서연
* 코드 작성일: 2025-04-17 ~
* 코드 작성 목적: LDA 토픽 모델링 구현

## 참조 코드 - Topic Modeling using LDA
* https://joyhong.tistory.com/138
* https://github.com/FifthSaint/NewsTextMining201903
* https://www.machinelearningplus.com/nlp/topic-modeling-visualization-how-to-present-results-lda-models/
* https://github.com/fastai/course-nlp/blob/master/2-svd-nmf-topic-modeling.ipynb

# 1. 기초 설정

## 사용 라이브러리

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import gensim
from gensim import corpora, models
from gensim.models import CoherenceModel

In [ ]:
import pyLDAvis
#import pyLDAvis.gensim
import pyLDAvis.gensim_models

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## 데이터 불러오기

In [ ]:
df_cleaned = pd.read_csv("../data/cleaned_docs.csv", encoding='utf-8-sig')
df_cleaned.info()

In [ ]:
terms = [doc.split() for doc in df_cleaned['cleaned_doc']]
terms[:5]

# 2. 전처리

## A. 사전과 문서-단어행렬 생성

In [ ]:
#training vocabulary
dictionary = corpora.Dictionary(terms)

In [ ]:
## 문서-단어 행렬(document-term matrix) 생성
corpus = [dictionary.doc2bow(term) for term in terms]

In [ ]:
print(dictionary)

## B. TF-IDF 행렬 계산

# 3. LDA 토픽 모델링 구현

## A. LDA 분석

In [ ]:
model = gensim.models.LdaModel(
    corpus,
    num_topics=8, # 출력 토픽 수
    id2word=dictionary,
    random_state=42
)

In [ ]:
# 각 토픽과 그 토픽에 속하는 단어를 중요도(확률)와 함께 출력
# num_topics: 출력할 토픽의 개수를 지정
# num_words: 각 토픽에서 상위 N개의 주요 단어 출력
print(model.show_topics(4, 10))

In [ ]:
NUM_TOPICS = 8 # dbpia 대분류(학문) 개수
TOP_N = 20

In [ ]:
word_dict = {}

In [ ]:
for i in range(NUM_TOPICS):
  words = model.show_topic(i, topn=TOP_N)
  word_dict['Topic #' + '{:02d}'.format(i+1)] = [i[0] for i in words]
  word_df = pd.DataFrame(word_dict)

In [ ]:
word_df

In [ ]:
model.print_topics(num_words=5)

In [ ]:
# 문서의 토픽 분포를 반환 = (토픽 번호, 확률)

for i in range(8):
    print(model.get_document_topics(corpus)[i])

## B. pyLDAvis를 이용한 시각화

In [ ]:
pyLDAvis.enable_notebook()

In [ ]:
#data = pyLDAvis.gensim.prepare(model, corpus, dictionary)
data = pyLDAvis.gensim_models.prepare(model, corpus, dictionary)

data

# 4. 하이퍼파라미터 조정

## A. LDA 토픽 개수 지정

In [ ]:
print('Perplexity: ', model.log_perplexity(corpus))
coherence_model_lda_1 = CoherenceModel(model=model, texts=terms, dictionary=dictionary, topn=10)
coherence_lda_1 = coherence_model_lda_1.get_coherence()
print('\nCoherence Score: ', coherence_lda_1)

### a. Perplexity - 낮을수록 좋음

In [ ]:
perplexity_values = []

In [ ]:
for i in range(2, 15):
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=i, id2word = dictionary, random_state=42)
    perplexity_values.append(ldamodel.log_perplexity(corpus))

In [ ]:
x = range(2, 15)
plt.plot(x, perplexity_values)
plt.title('Perplexity')
plt.xlabel("Number of topics")
plt.ylabel("Perplexity score")
plt.show()

### b. Coherence - 높을수록 좋음

In [ ]:
coherence_values = []

In [ ]:
for i in range(2, 15):
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=i, id2word=dictionary, random_state=42)
    coherence_model_lda = CoherenceModel(model=ldamodel, texts=terms, dictionary=dictionary, topn=10)
    coherence_lda = coherence_model_lda.get_coherence()
    coherence_values.append(coherence_lda)

In [ ]:
x = range(2, 15)
plt.plot(x, coherence_values)
plt.title('Coherence')
plt.xlabel("Number of topics")
plt.ylabel("coherence score")
plt.show()

최종적으로 Perplexity와 Coherence Score를 동시에 고려하여 최적의 주제 수를 선택

## B. 최종 LDA 분석

In [ ]:
# range(2, 15) -> 2부터이므로 index에 2를 더함
print(coherence_values.index(max(coherence_values)) + 2)

In [ ]:
NUM_TOPICS = coherence_values.index(max(coherence_values)) + 2

In [ ]:
final_model = gensim.models.LdaModel(
    corpus,
    num_topics=NUM_TOPICS, # 출력 토픽 수
    id2word=dictionary,
    random_state=42
)

In [ ]:
# 각 토픽과 그 토픽에 속하는 단어를 중요도(확률)와 함께 출력
# num_topics: 출력할 토픽의 개수를 지정
# num_words: 각 토픽에서 상위 N개의 주요 단어 출력

print(final_model.show_topics(4, 10))

In [ ]:
TOP_N = 20

In [ ]:
final_word_dict = {}
for i in range(NUM_TOPICS):
  final_words = final_model.show_topic(i, topn=TOP_N)
  final_word_dict['Topic #' + '{:02d}'.format(i+1)] = [i[0] for i in final_words]
  final_word_df = pd.DataFrame(final_word_dict)

In [ ]:
final_word_df

In [ ]:
print(final_model.print_topics(num_words=5))

In [ ]:
# 문서의 토픽 분포를 반환 = (토픽 번호, 확률)
for i in range(NUM_TOPICS):
    print(final_model.get_document_topics(corpus)[i])

## C. 최종 pyLDAvis 시각화

In [ ]:
pyLDAvis.enable_notebook()

In [ ]:
#data = pyLDAvis.gensim.prepare(model, corpus, dictionary)
final_data = pyLDAvis.gensim_models.prepare(final_model, corpus, dictionary)

final_data

In [ ]:
# HTML 파일로 저장
pyLDAvis.save_html(final_data, '../data/lda_vis.html')

# 5. 평가